**1. Libraries**

In [1]:
from requests_html import HTMLSession
import pandas as pd
import re, os, urllib, shutil
import pandas as pd
import sqlite3

**2. Directories**

In [2]:
## Creating directories
path_raw = os.path.join(os.getcwd(), 'Projections')
path_national = os.path.join(path_raw, 'National')
path_departmental = os.path.join(path_raw, 'Departmental')
path_municipal = os.path.join(path_raw, 'Municipal')
if os.path.exists(path_raw) == False: 
    os.mkdir(path_raw)
if os.path.exists(path_national) == False: 
    os.mkdir(path_national)
if os.path.exists(path_departmental) == False: 
    os.mkdir(path_departmental)
if os.path.exists(path_municipal) == False: 
    os.mkdir(path_municipal)

**3.2 Download and export**

In [3]:
def get_data(doc, pattern, path_level,csv_name, level):
    
    if level == 'Municipio':
        col_changes = {'DP':'Departamento', 'ÁREA GEOGRÁFICA':'Area', 'DPMP':'Municipio', 'AÑO':'Year'}
        column_names = ['Sex', 'Area']
        col_rem = ['Sexo_edad','DPNOM','MPIO']
    else:
        col_changes = {'DP':'Departamento', 'ÁREA GEOGRÁFICA':'Area', 'AÑO':'Year'}
        column_names = ['Sex', 'Area']
        col_rem = ['Sexo_edad','DPNOM']
        insert_projection = '''INSERT INTO Projections(departamento_id, area_id, sex_id, year, age, No_people) VALUES'''

    

    print('File:', doc)
    print('         Reading------------')
    new_all = pd.read_csv(doc)
    print(new_all.head(2))
    # Connecting with the database
    con = sqlite3.connect('Colombia_Demographic_data.sqlite', timeout=30)
    cur = con.cursor()
        
    z = 1
    if z == 1:
    ## area's table
        insert_area = 'INSERT INTO Area(name) VALUES'
        insert_area += '('
            
        Areas = new_all['Area'][new_all['Area'].notna()].unique()
        for area in Areas:
            area = area.capitalize()
            try:
                select_query = 'SELECT id FROM '
                select_query += 'Area' + ' WHERE name = ? ;'
                cur.execute(select_query, (area, ))
                cur.fetchone()[0]
                continue
            except:
                insert_area += '"'+str(area) + '"'+'),('
                    
        if insert_area != 'INSERT INTO Area(name) VALUES(':
            insert_area = insert_area[:-2] +';'
            cur.execute(insert_area)
            
        print('area inserted')
            
            ## Sex's table
        insert_sex = 'INSERT INTO Sexo(name) VALUES'
        insert_sex += '('
        for sex in new_all['Sexo'].unique():
            sex = sex.capitalize()
            try:
                select_query = 'SELECT id FROM '
                select_query += 'Sexo' + ' WHERE name = ? ;'
                cur.execute(select_query, (sex, ))
                cur.fetchone()[0]
                continue
            except:
                insert_sex += '"'+str(sex) + '"'+'),('
            
        if insert_sex != 'INSERT INTO Sexo(name) VALUES(':
            insert_sex = insert_sex[:-2] +';'
            cur.execute(insert_sex)
        con.commit()
        print('area and sex inserted')
        z += 1
            
    if level == 'Municipio':    
        insert_projection = '''INSERT INTO Projections(departamento_id, municipio_id, year, area_id, No_people, sex_id, age) VALUES'''
    else:
        insert_projection = '''INSERT INTO Projections(departamento_id, year , area_id, No_people, sex_id, age) VALUES'''

    r = 0
    for row in range(0,len(new_all)):
        
        i = 0
        insert_projection += '('
        for feature in new_all.iloc[row]:
                
            title   = new_all.columns.values[i]

            if title in column_names:
                feature = feature.capitalize()
                select_query = 'SELECT id FROM '
                select_query += str(title) + ' WHERE name = ? ;'
                cur.execute(select_query, (feature, ))
                try:
                    feature_id = cur.fetchone()[0]
                    insert_projection += '"'+str(feature_id) + '"'+','
                except:
                    print(title)
                    print(feature)
                    insert_projection += 'NULL,'
            elif pd.isna(feature):
                insert_projection += 'NULL,'

            else:
                insert_projection += '"' + str(feature) + '"'+','

            i +=1
        r +=1
        insert_projection = insert_projection[:-1] +'),'

        if r >= 10000:
            insert_projection = insert_projection[:-2] +');'
            cur.execute(insert_projection)
            
            r = 0
            if level == 'Municipio':    
                insert_projection = '''INSERT INTO Projections(departamento_id, municipio_id, year, area_id, No_people, sex_id, age) VALUES'''
            else:
                insert_projection = '''INSERT INTO Projections(departamento_id, year , area_id, No_people, sex_id, age) VALUES'''
            
    if r >0:
        insert_projection = insert_projection[:-2] +');'
        cur.execute(insert_projection)
        print('executed')
        con.commit()
        print('commited')
        
    con.close() 

In [4]:
get_data('Projections/National.csv', "Nac-[0-9]{4}-[0-9]{4}.xlsx$", path_national, 'Projections\\National.csv', 'x')

File: Projections/National.csv
         Reading------------
   DP   AÑO                               Area  No_personas     Sexo Edad
0   0  2020                           Cabecera     270528.0  Hombres    0
1   0  2020  Centros Poblados y Rural Disperso     117791.0  Hombres    0
area inserted
area and sex inserted
executed
commited


In [5]:
get_data('Projections/Departmental.csv', "dep-[0-9]{4}-[0-9]{4}.*\.xlsx$", path_departmental, 'Projections\\Departmental.csv', 'd')

File: Projections/Departmental.csv
         Reading------------


C:\Users\romer\AppData\Local\Temp\ipykernel_11772\168288192.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  new_all = pd.read_csv(doc)


   DP   AÑO                               Area  No_personas     Sexo Edad
0   5  1985                           Cabecera      26387.0  Hombres    0
1   5  1985  Centros Poblados y Rural Disperso      22738.0  Hombres    0
area inserted
area and sex inserted
executed
commited


In [ ]:
get_data(Municipal, "Mun-[0-9]{4}-[0-9]{4}.*\.xlsx$", path_municipal,'Projections\\Municipal.csv', 'Municipio')

**4. Remove directories**

In [ ]:
shutil.rmtree(path_national)
shutil.rmtree(path_departmental)
shutil.rmtree(path_municipal)